In [26]:
from keras.datasets import cifar10 
from random import sample 
import random
import numpy as np
import matplotlib.pyplot as plt

In [27]:
class Kmeans_Pointer:
    def __init__(self,x_point,y_point):
        self.x_point = x_point
        self.y_point = y_point
        self.size = x_point.size

In [28]:
def euclidean_distance(image1,image2):
    distance = np.sqrt(np.sum(np.square(image1-image2)))
    return distance

In [29]:
def clustering(points,center,k):
    clusters = [[] for i in range(k)]
    for point in points:
        distances = np.array([None]*k)
      
        for i in range(k):
            distances[i] = euclidean_distance(center[i].x_point,point.x_point)
            
        clus = np.argmin(distances)
        clusters[clus].append(point)
        
    centerA = [[] for i in range(k)]
    for index,cluster in enumerate(clusters):
        mean = []
        for i in range(3072):
            listX = [point.x_point[i] for point in cluster]
            mean.append(sum(listX)/len(listX))
        centerA[index] = Kmeans_Pointer(np.array(mean),0)
    
    total = True
    for old,new in zip(center,centerA):
        if not np.array_equal(old.x_point,new.x_point):
            total = False
            break
    if total == True:
        return centerA, clusters
    
    return clustering(points,centerA,k)

In [30]:
def k_mean(trainX,trainY,testX,testY,k):
    pixel_pointer = []
    predicted = []
    correct = 0
    incorrect = 0
    for x_pointer,y_pointer in zip(trainX,trainY):
        pixel_pointer.append(Kmeans_Pointer(x_pointer,y_pointer))
    center = pixel_pointer
    last_center,clusters = clustering(pixel_pointer,center,k)
   
    for x, the_cluster in enumerate(clusters):
        y_list = [point.y_point for point in the_cluster]
        y_mode = (max(set(y_list), key = y_list.count))
        last_center[x].y_point = y_mode
        
    for x in testX:
        distances = []
        for center in last_center:
            distances.append(euclidean_distance(x,center.x_point))
        tracker = np.argmin(np.array(distances))
        predicted.append(last_center[tracker].y_point)
        
    for m,n in zip(predicted,testY):
        if m == n[0]:
            correct += 1
        else:
            incorrect += 1
    return correct/(correct+incorrect)

In [31]:
def main():
    (trainX,trainY),(testX,testY) = cifar10.load_data()
    trainX = np.reshape(trainX, (trainX.shape[0], -1))
    testX = np.reshape(testX, (testX.shape[0], -1))
    
    ks = [3,6,9,12]
    Ktest = [0,0,0,0]
    for i in range(4):
        splitX = np.array_split(trainX[:50],5)
        splitY = np.array_split(trainY[:50],5)
        results = [[0,0] for i in range(5)]
        final_results = []
        for p in range(5):
            trainX_part = splitX[:p] + splitX[(p+1):]
            trainY_part = splitY[:p] + splitY[(p+1):]
            trainX_part = np.vstack(trainX_part)
            trainY_part = np.vstack(trainY_part).flatten()
            checkX = splitX[i]
            checkY = splitY[i]
            results = k_mean(trainX_part,trainY_part,checkX,checkY,ks[i])
        final_results.append(results)
           
        Ktest[i] = sum(final_results)/len(final_results)
        print("k",ks[i],"has a accuracy of ",Ktest[i]*100,"%\n")
        Kval = ks[Ktest.index(max(Ktest))]
        accuracy = 100*k_mean(trainX[:50],np.vstack(trainY).flatten(),testX[:50],testY,ks[Ktest.index(max(Ktest))])

    print("Best value of k was ",Kval," with a accuracy of ",accuracy,"%")
    
    return
    
if __name__ == "__main__":
    main()



k 3 has a accuracy of  30.0 %

k 6 has a accuracy of  20.0 %

k 9 has a accuracy of  30.0 %



ZeroDivisionError: division by zero